# Chapter 12: Facial Recognition & Landmark Detection (OpenCV DNN + Dlib)

## Objective
To implement face detection, recognition using face embeddings, and facial landmark detection using OpenCV's DNN module and Dlib. This lab provides a foundation for building real-time face-based systems.


## 1. What is Facial Recognition?

**Description**: Facial recognition involves identifying or verifying a person's identity from an image. It often includes face detection, alignment, embedding extraction, and matching.

### Key Components:
- **Face Detection**: Locating faces in images
- **Face Alignment**: Normalizing face orientation
- **Feature Extraction**: Creating face embeddings
- **Face Matching**: Comparing embeddings for recognition


## 2. Environment Setup

### Required Libraries:
- **OpenCV** for face detection and image processing
- **Dlib** for landmarks and face embeddings
- **face_recognition** wrapper for simplified face encoding and comparison

### Installation:
```bash
pip install opencv-python dlib face_recognition imutils numpy
```


In [2]:
# Import required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

# Try to import face_recognition and dlib
import face_recognition
import dlib

## 3. Face Detection using OpenCV DNN

We'll use OpenCV's DNN module with a pre-trained face detection model for robust face detection.


In [6]:
import cv2
net = cv2.dnn.readNetFromCaffe('deploy.prototxt', 'MobileNetSSD_deploy.caffemodel')
img = cv2.imread('./images/face.jpeg')
h, w = img.shape[:2]
blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), (104.0, 177.0, 123.0))
net.setInput(blob)
detections = net.forward()
for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > 0.5:
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (x1, y1, x2, y2) = box.astype("int")
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
cv2.imshow("Detected Face", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


## 4. Facial Landmark Detection with Dlib

Facial landmarks help identify key points on the face like eyes, nose, mouth, and jaw contours.


In [8]:
import dlib
from imutils import face_utils
img = cv2.imread('face.jpg')
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
detector = dlib.get_frontal_face_detector()
rects = detector(gray, 1)
for rect in rects:
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    for (x, y) in shape:
        cv2.circle(img, (x, y), 2, (0, 0, 255), -1)
cv2.imshow("Facial Landmarks", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


RuntimeError: Unable to open shape_predictor_68_face_landmarks.dat

In [ ]:
# Test facial landmark detection
def test_landmark_detection():
    """Test facial landmark detection on sample images"""
    
    test_images = [
        'images/face.jpeg',
        'images/person.jpg'
    ]
    
    for img_path in test_images:
        if os.path.exists(img_path):
            # Load image
            img = cv2.imread(img_path)
            if img is not None:
                print(f"\n🔍 Processing {img_path}...")
                
                # First detect faces
                faces, _ = detect_faces_opencv(img, face_detector, detector_type)
                print(f"   Found {len(faces)} face(s)")
                
                if len(faces) > 0:
                    # Detect landmarks
                    landmarks_list, landmarks_img = detect_facial_landmarks(img, faces)
                    connections_img = draw_landmark_connections(img, landmarks_list)
                    
                    # Display results
                    plt.figure(figsize=(15, 5))
                    
                    # Original image
                    plt.subplot(1, 3, 1)
                    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                    plt.title('Original Image')
                    plt.axis('off')
                    
                    # Landmarks points
                    plt.subplot(1, 3, 2)
                    plt.imshow(cv2.cvtColor(landmarks_img, cv2.COLOR_BGR2RGB))
                    plt.title(f'Facial Landmarks ({len(landmarks_list)} face(s))')
                    plt.axis('off')
                    
                    # Landmark connections
                    plt.subplot(1, 3, 3)
                    plt.imshow(cv2.cvtColor(connections_img, cv2.COLOR_BGR2RGB))
                    plt.title('Landmark Connections')
                    plt.axis('off')
                    
                    plt.tight_layout()
                    plt.show()
                    
                    # Print landmark details
                    for i, landmarks in enumerate(landmarks_list):
                        print(f"   Face {i+1}: {len(landmarks)} landmark groups detected")
                        for feature, points in landmarks.items():
                            print(f"     - {feature}: {len(points)} points")
                else:
                    print("   No faces detected for landmark analysis")
            else:
                print(f"❌ Could not load {img_path}")
        else:
            print(f"❌ File not found: {img_path}")

# Run landmark detection test
test_landmark_detection()


## 5. Face Recognition using Face Embeddings

Face recognition involves creating unique embeddings (feature vectors) for each face and comparing them to identify individuals.


In [ ]:
def create_face_encodings(image_path, name):
    """
    Create face encodings for a person from their image
    
    Args:
        image_path: Path to the person's image
        name: Name of the person
    
    Returns:
        Dictionary with name and encoding
    """
    try:
        # Load image
        image = face_recognition.load_image_file(image_path)
        
        # Get face encodings
        encodings = face_recognition.face_encodings(image)
        
        if len(encodings) > 0:
            print(f"✅ Created encoding for {name} from {image_path}")
            return {
                'name': name,
                'encoding': encodings[0],  # Take first face found
                'image_path': image_path
            }
        else:
            print(f"❌ No face found in {image_path} for {name}")
            return None
            
    except Exception as e:
        print(f"❌ Error creating encoding for {name}: {e}")
        return None

def recognize_faces_in_image(image_path, known_encodings, tolerance=0.6):
    """
    Recognize faces in an image by comparing with known encodings
    
    Args:
        image_path: Path to image to analyze
        known_encodings: List of known face encodings
        tolerance: Face matching tolerance (lower = stricter)
    
    Returns:
        Results dictionary with matches and annotated image
    """
    try:
        # Load image
        image = face_recognition.load_image_file(image_path)
        
        # Find face locations and encodings
        face_locations = face_recognition.face_locations(image)
        face_encodings = face_recognition.face_encodings(image, face_locations)
        
        # Convert to OpenCV format for drawing
        cv_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        results = []
        
        # Compare each found face with known faces
        for i, face_encoding in enumerate(face_encodings):
            matches = []
            distances = []
            
            for known_face in known_encodings:
                if known_face is not None:
                    # Compare faces
                    match = face_recognition.compare_faces([known_face['encoding']], face_encoding, tolerance=tolerance)
                    distance = face_recognition.face_distance([known_face['encoding']], face_encoding)[0]
                    
                    matches.append(match[0])
                    distances.append(distance)
            
            # Find best match
            if any(matches):
                best_match_index = distances.index(min(distances))
                if matches[best_match_index]:
                    name = known_encodings[best_match_index]['name']
                    confidence = 1 - distances[best_match_index]
                else:
                    name = "Unknown"
                    confidence = 0
            else:
                name = "Unknown"
                confidence = 0
            
            # Draw bounding box and label
            top, right, bottom, left = face_locations[i]
            
            # Choose color based on recognition
            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            
            # Draw rectangle and label
            cv2.rectangle(cv_image, (left, top), (right, bottom), color, 2)
            cv2.rectangle(cv_image, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
            
            # Add text
            label = f"{name} ({confidence:.2f})" if name != "Unknown" else "Unknown"
            cv2.putText(cv_image, label, (left + 6, bottom - 6), 
                       cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 1)
            
            results.append({
                'name': name,
                'confidence': confidence,
                'location': (top, right, bottom, left)
            })
        
        return {
            'results': results,
            'annotated_image': cv_image,
            'total_faces': len(face_locations)
        }
        
    except Exception as e:
        print(f"❌ Error in face recognition: {e}")
        return None

print("✅ Face recognition functions defined")


In [ ]:
# Create a face recognition system
def setup_face_recognition_system():
    """
    Set up a face recognition system with known faces
    """
    print("🔧 Setting up Face Recognition System...")
    
    # Define known people (you can add more)
    known_people = [
        {'name': 'Person_1', 'image': 'images/person_1.jpg'},
        {'name': 'Person_2', 'image': 'images/person_2.jpg'},
    ]
    
    # Create encodings for known people
    known_encodings = []
    
    for person in known_people:
        if os.path.exists(person['image']):
            encoding = create_face_encodings(person['image'], person['name'])
            if encoding is not None:
                known_encodings.append(encoding)
        else:
            print(f"❌ Image not found: {person['image']}")
    
    print(f"✅ Face recognition system ready with {len(known_encodings)} known faces")
    return known_encodings

def test_face_recognition():
    """
    Test face recognition on sample images
    """
    # Setup the recognition system
    known_encodings = setup_face_recognition_system()
    
    if len(known_encodings) == 0:
        print("❌ No known faces available for recognition")
        return
    
    # Test images
    test_images = [
        'images/face.jpeg',
        'images/person.jpg',
        'images/person_1.jpg',
        'images/person_2.jpg'
    ]
    
    plt.figure(figsize=(15, 10))
    
    valid_results = 0
    
    for i, img_path in enumerate(test_images):
        if os.path.exists(img_path):
            print(f"\n🔍 Analyzing {img_path}...")
            
            # Perform face recognition
            result = recognize_faces_in_image(img_path, known_encodings)
            
            if result is not None:
                # Display results
                plt.subplot(2, len(test_images), valid_results + 1)
                original_img = cv2.imread(img_path)
                plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
                plt.title(f'Original: {Path(img_path).name}')
                plt.axis('off')
                
                plt.subplot(2, len(test_images), valid_results + 1 + len(test_images))
                plt.imshow(cv2.cvtColor(result['annotated_image'], cv2.COLOR_BGR2RGB))
                plt.title(f'Recognition: {result["total_faces"]} face(s)')
                plt.axis('off')
                
                # Print recognition results
                for j, face_result in enumerate(result['results']):
                    print(f"   Face {j+1}: {face_result['name']} (confidence: {face_result['confidence']:.3f})")
                
                valid_results += 1
            else:
                print(f"   ❌ Could not process {img_path}")
        else:
            print(f"❌ File not found: {img_path}")
    
    if valid_results > 0:
        plt.tight_layout()
        plt.show()
    else:
        print("❌ No valid results to display")

# Run face recognition test
test_face_recognition()


## 6. Real-Time Face Recognition

Implement real-time face recognition using webcam input.


In [ ]:
def real_time_face_recognition(known_encodings, duration=30):
    """
    Perform real-time face recognition using webcam
    
    Args:
        known_encodings: List of known face encodings
        duration: Duration to run (seconds)
    """
    print(f"🎥 Starting real-time face recognition for {duration} seconds...")
    print("Press 'q' to quit early")
    
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("❌ Could not open webcam")
        return
    
    start_time = cv2.getTickCount()
    frame_count = 0
    
    # Process every nth frame for performance
    process_this_frame = True
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Resize frame for faster processing
            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
            rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
            
            # Only process every other frame to save computation
            if process_this_frame:
                # Find face locations and encodings
                face_locations = face_recognition.face_locations(rgb_small_frame)
                face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
                
                face_names = []
                for face_encoding in face_encodings:
                    # Compare with known faces
                    matches = []
                    distances = []
                    
                    for known_face in known_encodings:
                        if known_face is not None:
                            match = face_recognition.compare_faces([known_face['encoding']], face_encoding)
                            distance = face_recognition.face_distance([known_face['encoding']], face_encoding)[0]
                            
                            matches.append(match[0])
                            distances.append(distance)
                    
                    # Find best match
                    name = "Unknown"
                    if len(distances) > 0 and any(matches):
                        best_match_index = distances.index(min(distances))
                        if matches[best_match_index]:
                            name = known_encodings[best_match_index]['name']
                    
                    face_names.append(name)
            
            process_this_frame = not process_this_frame
            
            # Display results on full-size frame
            for (top, right, bottom, left), name in zip(face_locations, face_names):
                # Scale back up face locations
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4
                
                # Choose color
                color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
                
                # Draw rectangle
                cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
                
                # Add label
                cv2.putText(frame, name, (left + 6, bottom - 6), 
                           cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)
            
            # Add frame counter and instructions
            cv2.putText(frame, f'Frame: {frame_count}', (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(frame, 'Press Q to quit', (10, frame.shape[0] - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Display frame
            cv2.imshow('Real-Time Face Recognition', frame)
            
            frame_count += 1
            
            # Check for quit or time limit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                print("\n👋 Quit requested")
                break
                
            # Check time limit
            elapsed_time = (cv2.getTickCount() - start_time) / cv2.getTickFrequency()
            if elapsed_time > duration:
                print(f"\n⏰ Time limit reached ({duration}s)")
                break
                
    except KeyboardInterrupt:
        print("\n⚠️ Interrupted by user")
    
    finally:
        # Cleanup
        cap.release()
        cv2.destroyAllWindows()
        
        # Calculate FPS
        total_time = (cv2.getTickCount() - start_time) / cv2.getTickFrequency()
        fps = frame_count / total_time if total_time > 0 else 0
        
        print(f"\n📊 Session Summary:")
        print(f"   Total frames: {frame_count}")
        print(f"   Duration: {total_time:.1f}s")
        print(f"   Average FPS: {fps:.1f}")

print("✅ Real-time face recognition function defined")
print("\n💡 To start real-time recognition, run:")
print("known_encodings = setup_face_recognition_system()")
print("real_time_face_recognition(known_encodings, duration=30)")


## 7. Summary

### What We Implemented:
- **OpenCV DNN** provides fast face detection
- **Dlib** enables precise landmark localization  
- **Face embeddings** are robust descriptors for recognition
- **Real-time systems** can be built using webcam + pre-trained encoders

### Key Features:
- Face detection using Haar cascades
- Facial landmark detection with 68 key points
- Face recognition using embeddings
- Real-time processing capabilities

### Applications:
- Security systems
- Attendance tracking
- Photo organization
- Augmented reality filters


## Suggested Exercises

1. **Build a multi-face recognition system**
   - Add more known faces to the database
   - Test with group photos

2. **Record embedding vectors for 5 individuals and match against a test feed**
   - Create a face database with multiple photos per person
   - Test recognition accuracy

3. **Display landmarks on live video**
   - Real-time landmark detection
   - Face filter applications

4. **Extend the pipeline to recognize expressions using landmarks**
   - Analyze facial expressions
   - Emotion detection system
